# ``orsopy`` Integration

``python-RAT`` contains some integration with ``orsopy``, allowing for convenient interaction with the ``.ort`` file format. This integration is available through the `RATapi.utils.orso` submodule.

In [8]:
import RATapi.utils.orso

## Creating models from the ORSO model description language

The [ORSO model description format](https://www.reflectometry.org/advanced_and_expert_level/file_format/simple_model) allows the description of a standard slab model as a one-line string, provided that all the layer materials are defined [in the ORSO SLD database](https://slddb.esss.dk/slddb/).

The function `RATapi.utils.orso.orso_model_to_rat` function can read a model and return an `ORSOSample` dataclass, which gives bulk in and bulk out parameters for the model, a list of all layers defined in the model, and all the parameters needed to define those layers as RAT models. 

**Note:** the ORSO format gives the thicknesses of materials in *nanometres*. When we convert them to RAT parameters, the units will be converted to Angstroms.

For example, the string `air | Ni 100 | SiO2 0.5 | Si` describes a 1000 angstrom nickel film backed by a 5 angstrom silicon oxide layer. The bulk-in and bulk-out are air and silicon respectively. The roughnesses and SLDs will be calculated or taken from the ORSO SLD database.

In [ ]:
# create the RAT parameters and layers from this model
sample = RATapi.utils.orso.orso_model_to_rat("air | Ni 100 | SiO2 0.5 | Si")
print("Bulk in:\n", sample.bulk_in, "\nBulk out:\n", sample.bulk_out)
print("Parameters:\n", sample.parameters)
print("Layers:\n", sample.layers)

You can also set `absorption=True` and the model will account for absorption. For example if we change the nickel film for a boron carbide film and want to account for its relatively high absorption, we can add it to the output:

In [ ]:
sample = RATapi.utils.orso.orso_model_to_rat("vacuum | B4C 100 | SiO2 0.5 | Si", absorption=True)
print("Parameters:\n", sample.parameters)
print("Layers:\n", sample.layers)

Finally, ORSO supports defining repeated layers using parentheses. For example, if we had a polarising multilayer of 5 repetitions of 70 angstrom silicon and 70 angstrom iron, we could represent it as `air | 5 ( Si 7 | Fe 7 ) | Si`.

RAT will only create the number of layers and parameters necessary, but the `ORSOSample` object's `model` attribute will give a list of layer names with the structure of the model preserved, which can be given as the layer model for a Contrast.

In [ ]:
sample = RATapi.utils.orso.orso_model_to_rat("air | 5 ( Si 7 | Fe 7 ) | Si")
print("Parameters:\n", sample.parameters)
print("Layers:\n", sample.layers)
print("Model:\n", sample.model)

## Reading in data and models from .ort files

It is possible to read data from .ort format files at two different levels of depth.

### Reading data

Firstly, if you just want to read the data from a file, you can do so via the function `RATapi.utils.orso.load_ort_data`.

This will read the .ort data in as a RAT `Data` object. If there are multiple datasets in your file, you will get a list of datasets back.

In [ ]:
import pathlib
data_path = pathlib.Path("../data")

data = RATapi.utils.orso.load_ort_data(data_path / "IvsQ_82043_82044.ort")
print(data)

### Reading an entire project
If your .ort file contains model data, you can also use the more elaborate `RATapi.utils.orso.ort_to_project`, which creates an almost-complete RAT `Project` with model and data from the file. If your file contains multiple datasets with models, a contrast will be created in the project for each model.

Note that none of the parameters will have ranges or be fittable (you will have to manually change the minimum and maximum range if you want to fit the parameters!). Furthermore, the background and scalefactor are not part of the format and will just be empty placeholders in this project. 

In [ ]:
project = RATapi.utils.orso.ort_to_project(data_path / "prist5_10K_m_025.Rqz.ort")
print(project)

In [ ]:
controls = RATapi.Controls()
project, results = RATapi.run(project, controls)
RATapi.plotting.plot_ref_sld(project, results)